In [23]:
using DataFrames, DataFramesMeta, LinearAlgebra, CSV, Distributions, Optim, Revise, Plots

In [56]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule

In [57]:
# I = 100;              # Number of consumers
J = 3;                  # Number of firms
K = 1;                  # Product characteristics
nT = 5;                 # Number of markets
# β = [1, -1];            # Preferences
β = -1.;            # Preferences
varζ = 3;               # Variance of the random taste
# rangeJ = [J, J];        # TODO: set a min and max firms per market
rangeB = [100, 1000];       # consumers per market (e.g. census block)
# varX = 2;               # Variance of X
varξ = 1;               # Variance of xi

In [75]:
dist_thresh = 1.
df = BLPmodule.build_dist_data(J, β, nT, rangeB, varζ, varξ, dist_thresh)
df[:, Cols("t", "j", r"q", r"B", r"agg")]

,t,j,q,q0,q_j,agg_q0,B,total_B,agg_s,agg_s0
,Int64,Int64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64
1,1,1,144.136,192.472,647.868,781.837,638,2685,0.241292,0.291187
2,1,2,142.033,192.472,637.475,781.837,638,2685,0.237421,0.291187
3,1,3,159.358,192.472,617.82,781.837,638,2685,0.230101,0.291187
4,2,1,149.034,163.562,647.868,781.837,576,2685,0.241292,0.291187
5,2,2,136.435,163.562,637.475,781.837,576,2685,0.237421,0.291187
6,2,3,126.968,163.562,617.82,781.837,576,2685,0.230101,0.291187
7,3,1,46.1161,56.2573,647.868,781.837,191,2685,0.241292,0.291187
8,3,2,46.8223,56.2573,637.475,781.837,191,2685,0.237421,0.291187
9,3,3,41.8043,56.2573,617.82,781.837,191,2685,0.230101,0.291187


In [90]:
# Simulate
# df = BLPmodule.simulate_data(I, J, β, nT, rangeJ, rangeB, varζ, varX, varξ)
# Retrieve data
T = Int.(df.t)
X_ = df.dist
# s_ = df.s
q_j_ = df.q_j
s0_ = df.s0
q0_ = df.q0
# IV_ = [df.x1 df.w]
# Z_ = [df.x1 df.z1]
nI = 100
# ζ_ = [rand(Normal(0,1), 1, nI); zeros(K, nI)]; # Draw shocks (less)
ζ_ = rand(Normal(0,1), 1, nI); # Draw shocks (less)

In [91]:
# using BLPmodule
# Minimize GMM objective function
# varζ_ = optimize(x -> BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, x[1])[1], [3.0], LBFGS()).minimizer[1];

In [92]:
agg_q = [q_j_[df.j .== j][1] for j in 1:J]
total_outside = sum(unique(df.q0))
agg_q0 = repeat([total_outside], J)
total_B = sum([sum(agg_q), total_outside])
agg_shares = agg_q ./ total_B
agg_s0 = agg_q0 ./ total_B;

In [97]:
t1ind = df.t .== 1;

In [99]:
using BLPmodule
# deltas = BLPmodule.loop(agg_shares, agg_s0, X_[t1ind], ζ_, T);

In [102]:
using BLPmodule
deltas = BLPmodule.compute_delta(agg_shares, agg_s0, X_[t1ind], ζ_, T[t1ind]);

deltas:

[-0.21506138455491342, -0.2959378069473715, -0.22835075082107825]


In [103]:
δ0 = zeros(1, size(ζ_, 2))
shares = BLPmodule.implied_shares(X_[t1ind,:], ζ_, deltas, δ0)
println("computed shares:       ", shares)
println("true aggregate shares: ", agg_shares)

computed shares:       [0.24129171695127144, 0.2374209060593391, 0.23010053446712916]
true aggregate shares: [0.24129171557972579, 0.2374209046837222, 0.23010053319031587]


In [62]:
# δ0 = zeros(1, size(ζ_, 2))
# for t in unique(T)
#     tind = df.t.==t
#     shares = BLPmodule.implied_shares(X_[tind,:], ζ_, deltas1, δ0)
#     println("computed shares: ", shares)
# end
# println(agg_shares)

In [63]:
# β_blp = BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, varζ_)[2];

In [64]:
# print("Estimated BLP coefficients: $β_blp")

In [65]:
# # Compute logit estimate
# y = log.(df.s) - log.(df.s0)
# β_logit = inv(IV_' * X_) * (IV_' * y)
# print("Estimated logit coefficients: $β_logit")